In [1]:
import cv2
import numpy as np

# Load Yolo
print("LOADING YOLO")
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
#save all the names in file o the list classes
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
#get layers of the network
layer_names = net.getLayerNames()
#Determine the output layer names from the YOLO model 
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print("YOLO LOADED")

LOADING YOLO
YOLO LOADED


In [20]:
# Capture frame-by-frame
img = cv2.imread("living_room.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# USing blob function of opencv to preprocess image
blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), (0,0,0),
    swapRB = True, crop = False)

#Detecting objects
net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
    
#We use NMS function in opencv to perform Non-maximum Suppression
#we give it score threshold and nms threshold as arguments.
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
colors = np.random.uniform(0, 255, size=(len(classes), 3))
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        print(label)
        color = colors[class_ids[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y -5),cv2.FONT_HERSHEY_SIMPLEX, 1/2, color, 2)

cv2.imshow("Image",img)
cv2.resizeWindow("Image", 416, 416)
cv2.waitKey(0)

tvmonitor
person
dog
chair


-1

In [19]:
start_mid_x = (width/2) - (width/6)
end_mid_x = (width/2) + (width/6)

start_mid_y = (height/2) - (height/6)
end_mid_y = (height/2) + (height/6)

print("start_mid_x = " + str(start_mid_x))
print("end_mid_x = " + str(end_mid_x))

print("start_mid_y = " + str(start_mid_y))
print("end_mid_y = " + str(end_mid_y))

text = 'dog'
obj_class_index = classes.index(text)

for i in range(len(boxes)):
    if i in indexes:
        if obj_class_index == class_ids[i]:
            x, y, w, h = boxes[i]
            
            midx = x + w/2
            midy = y + h/2
        
            print(str(classes[class_ids[i]]))
            print("x = " + str(x))
            print("y = " + str(y))
            print("w = " + str(w))
            print("h = " + str(h))
            
            print("midx = " + str(midx))
            print("midy = " + str(midy))
        
            if start_mid_x <= midx <= end_mid_x:
                if start_mid_y<= midy <=  end_mid_y:
                    print("in the center")
                elif midy < start_mid_y:
                    print("bottom center")
                else:
                    print("top center")

            elif midx < start_mid_x:
                if midy < start_mid_y:
                    print("bottom left")
                elif midy > end_mid_y:
                    print("top left")
                else:
                    print("center left")

            else:
                if midy < start_mid_y:
                    print("bottom right")
                elif midy > end_mid_y:
                    print("top right")
                else:
                    print("center right")
                
        
        

start_mid_x = 80.0
end_mid_x = 160.0
start_mid_y = 70.66666666666666
end_mid_y = 141.33333333333334
dog
x = 118
y = 132
w = 50
h = 52
midx = 143.0
midy = 158.0
top center


In [15]:
class_ids

[62, 0, 0, 0, 0, 16, 16, 16, 56, 56, 56, 16, 16]

In [30]:
len(boxes)

13

In [4]:
import torch

#model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

Using cache found in C:\Users\Narde/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\Narde/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(144,

In [6]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in C:\Users\Narde/.cache\torch\hub\intel-isl_MiDaS_master


In [7]:
input_batch = transform(img).to(device)

with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

output = prediction.cpu().numpy()

In [9]:
from sklearn.preprocessing import MinMaxScaler
print(output)
scaler = MinMaxScaler(feature_range = (-1,1))
output = scaler.fit_transform(output)
print(output)

[[ 52.46621   48.99262   48.909336 ... 113.67614  109.70239   93.096985]
 [ 50.812138  50.478558  49.53688  ... 111.4928   112.70707  106.49761 ]
 [ 48.80058   49.567726  49.537273 ... 112.26802  112.37253  110.189255]
 ...
 [926.3711   907.9527   900.73895  ... 827.11066  858.64136  874.5412  ]
 [942.7446   935.65186  916.11646  ... 849.2082   880.96826  886.8016  ]
 [930.8094   928.4505   920.1119   ... 847.62866  885.883    875.5974  ]]
[[-0.991799   -0.99873024 -0.99923897 ... -0.95950043 -0.9774207
  -1.        ]
 [-0.9954996  -0.9953806  -0.99779886 ... -0.965317   -0.9697659
  -0.9662327 ]
 [-1.         -0.99743384 -0.99779797 ... -0.9632517  -0.9706181
  -0.9569304 ]
 ...
 [ 0.96336794  0.93755984  0.9555429  ...  0.9411309   0.9305985
   0.96910584]
 [ 1.          1.          0.9908315  ...  1.          0.9874792
   0.9999999 ]
 [ 0.9732976   0.98376656  1.0000001  ...  0.9957919   1.
   0.9717673 ]]


In [10]:
depth_limit = 10
x_diff = bottom_right[0] - top_left[0]
print(x_diff)
ixgrid = np.ix_( range(top_left[1] , bottom_right[1]) , range(top_left[0] , bottom_right[0] ))
print(output[ixgrid].shape)
depth_image = np.sum(output[ixgrid])
print(depth_image)

if depth_image < depth_limit:
  text += ' and not near'
  print('not near')
else:
  text += 'and near'
  print('near')

NameError: name 'bottom_right' is not defined